In [14]:
import json
import os
import pprint
import random
from typing import List, Dict
import uuid
import yaml

from jinja2 import Template, StrictUndefined
from openai import OpenAI, AsyncOpenAI
from pydantic import BaseModel, Extra, Field
import tiktoken

from src.models import CharacterSpecification
from config import settings

In [10]:
# Initialize OpenAI provider
print(settings.llm_model)
# OpenAI Client
llm_client = OpenAI(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)

gpt-4.1-nano


# 1. Select 2 Characters

In [3]:
with open("assets/character_collection.json", "r") as f:
    character_collection = json.load(f)

character_ids = list(character_collection["source"].keys())

In [4]:
# fix characters for testing
character1_id="35f0c56f-263d-42df-846c-e1833d8ca0ab"
character2_id="00d66087-9b3b-46da-bd74-bf45cbe81d3c"

print(character1_id, character_collection["source"][character1_id])
print(character2_id, character_collection["source"][character2_id])

35f0c56f-263d-42df-846c-e1833d8ca0ab wiki_Zephyr Orion.txt
00d66087-9b3b-46da-bd74-bf45cbe81d3c wiki_Vivienne LaRoux.txt


In [5]:
# Load Characters
with open(f"assets/characters/{character1_id}.json", "r") as f:
    character1_spec = CharacterSpecification(**json.load(f))
    
with open(f"assets/characters/{character2_id}.json", "r") as f:
    character2_spec = CharacterSpecification(**json.load(f))

# 2. Generate Scene Draft

In [6]:
with open('prompts/scene_initialization.yaml', 'r') as file:
    prompt = yaml.load(file, Loader=yaml.FullLoader)

In [11]:
system_message = prompt['system']
user_template = Template(
    prompt['user'],
    undefined=StrictUndefined
)

class InitTropeResult(BaseModel):
    trope: str
    class Config:
        extra = Extra.forbid

def generate_initial_trope(
    character1: CharacterSpecification,
    character2: CharacterSpecification,
):
    user_message = user_template.render(
        characters=[character1, character2]
    )
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    decode_params = {"temperature": 0.95}

    response = llm_client.beta.chat.completions.parse(
        model=settings.llm_model,
        messages=messages,
        response_format=InitTropeResult,
        **decode_params,
    )
    return response.choices[0].message.parsed.trope

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_17217/4109169911.py:10: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [12]:
initial_trope=generate_initial_trope(
    character1_spec,
    character2_spec
)
# Example (for testing)
# initial_trope="In the glamorous hustle and bustle of a metropolitan fashion gala, **Zephyr Orion**, a 28-year-old social magnet with an astronaut's bravado, finds himself drawn to the assertive allure of **Vivienne LaRoux**, a renowned 28-year-old style influencer. Standing out in the crowd with his easy-going charm, Zephyr playfully engages Vivienne in a conversation, his jocular tone a stark contrast to her sharply assertive style. She, poised and confident, initially dismisses Zephyr's witty banter with an elegant roll of the eyes, but his relentless charisma gradually cracks her facade, revealing a flicker of intrigued amusement. Despite Vivienne's mean streak occasionally surfacing, the evening unfolds into a dynamic exchange, Zephyr's warmth melting Vivienne's icy exterior as they navigate through complex social dance, caught between playful banter and subtle challenges to each other's worldviews."

In [15]:
pprint.pprint(initial_trope)

('Zephyr Orion, the outgoing and ambitious astronaut with a playful, jovial '
 'manner, loves to tell captivating tales of space adventures that make '
 'everyone around him feel at ease. Vivienne LaRoux, a sharp and assertive '
 'style influencer with a chic but occasionally dismissive attitude, often '
 'changes her opinions on fashion and lifestyle with a noncommittal air. In '
 'this small scene, Zephyr, eager to share his latest space story, quickly '
 "realizes that Vivienne's sharp wit and high standards don't easily lend "
 'themselves to his lighthearted storytelling, leading to a teasing exchange '
 'that reveals her dismissive side and his need to impress, creating a playful '
 'yet slightly tense dynamic.')


# 3. Refine 

## 3-1. Retrieve Similar Tropes

In [26]:
from qdrant_client import QdrantClient
from llama_index.core import (
	Document,
	SimpleDirectoryReader, VectorStoreIndex,
	StorageContext,
	Settings,
	QueryBundle
)

from llama_index.embeddings.text_embeddings_inference import (
    TextEmbeddingsInference,
)

from src.tropes.retriever import TropeRetriever
from src.tropes.models import Trope

In [22]:
embedder = TextEmbeddingsInference(
    settings.embedding_model,
    base_url=settings.embedding_base_url,
    auth_token=settings.embedding_api_key
)
Settings.embed_model = embedder
client = QdrantClient(host="localhost", port=settings.qdrant_port)

retriever = TropeRetriever(client)

In [23]:
retrieved_tropes = retriever.retrieve(initial_trope)
print(len(retrieved_tropes))

5


In [24]:
for trope in retrieved_tropes:
    print(trope)

trope_id='t13173' name='LoonyFriendsImproveYourPersonality' explanation='A stiff character learns and grows from unwanted interaction with annoying and eccentric people.\nThe focus character is a fairly stiff guy (rarely a gal), may be the Only Sane Man, The Comically Serious or a Jerkass of the "stick up his rear" type, and often comes across as an Ineffectual Loner. He may not be content with his life, but it\'s stable, and he probably has a long-term plan for fixing what he thinks is wrong, if he can just get the right breaks. Instead, he is dragged into wacky hijinks by the other characters against his will, making a mess of his life. But implicitly or explicitly, the goal in the story is to make him a better person by putting him through "horrible" experiences - or at least ones he considers horrible, in order to show him the brighter side of things.\nMay involve a Manic Pixie Dream Girl, but in most examples, it\'s an entire cast imposing on the stiff guy\'s time, money and patie

## 3-2. Refine with retrieved

In [25]:
def format_trope(trope):
    return "{{\"name\": {name}, \"explanation\": {explanation}}}".format(
        name=trope.name, explanation=trope.explanation
    )
    
class SceneState(BaseModel):
    location: str
    setting: str
    explanation: str
    
    class Config:
        extra = Extra.forbid
        use_enum_values = True

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_17217/1847138306.py:12: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [27]:
with open('prompts/scene_refinement.yaml', 'r') as file:
    prompt = yaml.load(file, Loader=yaml.FullLoader)

In [30]:
system_message = prompt['system']
user_template = Template(
    prompt['user'],
    undefined=StrictUndefined
)

class RefineTropeOutput(BaseModel):
    revised_trope: str
    utilized_reference_tropes: List[str]
    scene: SceneState
    
    class Config:
        extra = Extra.forbid
        use_enum_values = True

def refine_trope(
    character1: CharacterSpecification,
    character2: CharacterSpecification,
    draft: str,
    reference_tropes: List[Trope]
    
):
    user_message = user_template.render(
        characters=[character1, character2],
        draft=draft,
        reference_tropes=reference_tropes
    )
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    decode_params = {"temperature": 0.95}

    response = llm_client.beta.chat.completions.parse(
        model=settings.llm_model,
        messages=messages,
        response_format=RefineTropeOutput,
        **decode_params,
    )
    return response.choices[0].message.parsed

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_17217/2820334285.py:13: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [31]:
refined_trope = refine_trope(
    character1_spec,
    character2_spec,
    draft=initial_trope,
    reference_tropes=retrieved_tropes
)

In [32]:
print(refined_trope.model_dump_json(indent=4))

{
    "revised_trope": "During a casual but lively social gathering in Zephyr Orion's modern city apartment, Zephyr, the outgoing, jovial astronaut and known for his storytelling, is attempting to impress Vivienne LaRoux, the confident and stylish style influencer. Zephyr, with his playful, goofy personality, starts sharing a humorous, exaggerated space adventure story, knowing Vivienne's sharp, assertive nature and love for intellectual engagement. Vivienne, initially dismissive and sharp, challenges Zephyr with a quick, sarcastic retort, testing his wit and patience, but also secretly intrigued by his ease and charm. As Zephyr responds with humor and warmth, their interaction reveals a subtle evolution from mutual teasing to a moment of genuine connection, showcasing Zephyr's effort to break through Vivienne's dismissive exterior with his playful sincerity.",
    "utilized_reference_tropes": [
        "LoonyFriendsImproveYourPersonality"
    ],
    "scene": {
        "location": "Zep